In [1]:
class SpreadFunction:
    @staticmethod
    def generate_N_s_i(p_s, i):
        """Generates the value of N_s_i according to the original paper.
           N_s_i = 1/(2*p_s) + i/p_s

        Args:
            p_s (float): the probability of a specific symbol
            i (int): the index of the N_s value we want to get

        Returns:
            float: returns the N_s_i value
        """
        return 1/(2*p_s) + i/p_s

    @staticmethod
    def generate_N_s(p_s, l):
        """Generates N_s for a specific symbol with a given length.
           N_s = [N_s_0, N_s_1, ..., N_s

        Args:
            p_s (float): the probability of a specific symbol
            l (int): length of the N_s we want to generate

        Returns:
            list: list of N_s values
        """
        return [SpreadFunction.generate_N_s_i(p_s, i) for i in range(l)]

    @staticmethod
    def generate_N(prob, l):
        """Generates N for a given set of symbols, given their probabilities and the length of the N_s values.

        Args:
            prob (list): list of probabilities of the symbols
            l (int): length of the N_s values we want to generate

        Returns:
            list of lists: list of N_s values for each symbol
        """
        return [SpreadFunction.generate_N_s(p, l) for p in prob]

    @staticmethod
    def generate_N_iter(prob, l):
        """Generates a list of tuples with the N values and their indexes, sorted in ascending order.
           This is used to iterate over the N values in the order they should be used, as defined in the original paper.

        Args:
            prob (list): list of probabilities of the symbols
            l (int): length of the N_s values we want to generate

        Returns:
            list of lists: list of tuples with the N values and their indexes, sorted in ascending order with respect to the N values
        """
        N = SpreadFunction.generate_N(prob, l)
        N_iter = [[N[i][j], i] for i in range(len(N)) for j in range(len(N[i]))]
        N_iter.sort()
        return N_iter

In [2]:
import numpy as np 
import math
import pandas as pd
# Building Decoding table

class DecodeTableColumn:
    def __init__(self, x, sym, nbBits, newX):
        """Initializes the decoding table column

        Args:
            x (int): the state of the column
            sym (void): the symbol for that column, can be of any type
            nbBits (int): number of bits to read from bitstream
            newX (int): new state after reading nbBits
        """
        self.x = x
        self.sym = sym
        self.nbBits = nbBits
        self.newX = newX
        
    def __str__(self):
        return "State: %d, Symbol: %s, nbBits: %d, newX: %d" % (self.x, self.sym, self.nbBits, self.newX)
    
    def __list__(self):
        return [self.x, self.sym, self.nbBits, self.newX]
        
class DecodeTable:
    def __init__(self, L, s_list, L_s, fast = False):
        """Initializes the decoding table

        Args:
            L (int): the length of the table, equal to the culminative frequency of the symbols
            s_list (list): list of symbols in the message, for example (A,B,C)
            L_s (list): the frequency of each symbol in the message, for example (2,3,4)
        """
        
        self.L = L
        self.s_list  = s_list
        self.L_s = L_s
        
        # initialize an empty table, with L empty columns
        self.table = [DecodeTableColumn(0,0,0,0) for i in range(L)]
        
        # spread symbols using the spread function
        self.symbol_spread = None
        if fast:
            self.fast_spread(X = 0, step = int((5/8) * self.L + 3))
        else:
            self.spread_function()
        
        # generate the decoding table
        self.get_decoding_table()
        
        
    def __str__(self):
        return "Decoding table with length: %d, symbols: %s, frequencies: %s" % (self.L, self.s_list, self.L_s)
    
    def display_table(self):
        """Generates a pandas dataframe to display the decoding table

        Returns:
            pd.DataFrame: a pandas dataframe with the decoding table
        """
        
        # make a list of lists for the table
        table = []
        for i in range(self.L):
            table.append(self.table[i].__list__())
            
        # convert to pandas dataframe
        df = pd.DataFrame(table, columns=["State", "Symbol", "nbBits", "newX"])
        df.set_index("State", inplace=True)
        return df.T

    def spread_function(self):
        """Uses the heuristic spread function to spread the symbols in the decoding table

        Returns:
            list: a list of symbols spread according to the spread function
        """
        
        symbol = [0 for i in range(self.L)]
        spread = SpreadFunction.generate_N_iter([i/self.L for i in self.L_s], self.L)
        
        for X, e in enumerate(spread):
            if X < self.L:
                symbol[X] = e[1]
            else:
                break
                
        # update in place
        self.symbol_spread = symbol
        return symbol
    
    def fast_spread(self, X, step):
        symbol = [0 for i in range(self.L)]
        for s in range(len(self.s_list)):
            for i in range(1, self.L_s[s]+1):
                symbol[X] = s
                X = (X + step) % self.L
                
        self.symbol_spread = symbol
        return symbol
    
    @staticmethod
    def read_bit(bitstream, nbits):
        """Read nbits from bitstream (a list of bits), and return the bits read and the remaining bitstream.
           Reads from right to left.

        Args:
            bitstream (list): the binary bitstream to read from (list of 0s and 1s)
            nbits (int): number of bits to read

        Returns:
            (list, list): returns (bits, bitstream) where bits is the list of bits read, and bitstream is the remaining bitstream
        """
        # check if nbits == 0
        if nbits == 0:
            return [], bitstream
        # read nbits from bitstream
        bits = bitstream[-nbits:]  # get the last nbits
        bitstream = bitstream[:-nbits]  # remove the last nbits from the bitstream
        return bits, bitstream

    def get_decoding_table(self):
        """Generates the decoding table, and returns. Also updates table in place.

        Returns:
            list of lists: returns the decoding table as a list of lists
        """
        
        # next holds the next x_tmp for each symbol and is incremented after each use
        # is initialized to the frequency of each symbol
        next = [e for e in self.L_s] 
        
        # calculate R, the number of bits needed to represent L
        R = math.log2(self.L) # L = 2^R
        
        # generate the decoding table
        for X in range(self.L):
            # set the x value
            self.table[X].x = X
            
            # set the symbol, given by the spread function
            self.table[X].sym = self.symbol_spread[X]

            # calculate x_tmp
            x_tmp = next[self.symbol_spread[X]]
            
            # increment next at the index of the symbol
            next[self.symbol_spread[X]] += 1
            
            # calculate the number of bits needed to represent x_tmp
            self.table[X].nbBits = int(R - math.floor(math.log2(x_tmp)))
            
            # calculate the new x value
            self.table[X].newX = (x_tmp << self.table[X].nbBits) - self.L
        
        return self.table
    
    def decode_step(self, state, bitstream):
        """A single step in the decoding process, decodes a single symbol from the bitstream

        Args:
            state (int): the current state of the decoding process, in the range [0, L)
            bitstream (list): A bitstream to decode, should be a list of 0s and 1s

        Returns:
            (void, int, list): returns (decoded symbol, next state, remaining bitstream)
        """
        
        # get the symbol
        s_decode = self.table[state].sym
        
        # read into bits from the bitstream, and update the stream back into bitstream
        # bits is the bits read from the bitstream
        # bitstream is the remaining bitstream after reading bits
        bits, bitstream = DecodeTable.read_bit(bitstream, self.table[state].nbBits)
        if len(bits) == 0:
            bits = 0
        else:
            bits = int("".join(str(i) for i in bits), 2) # convert bits to int
        
        # calculate the next state
        next_state = self.table[state].newX + bits
        
        return s_decode, next_state, bitstream
    
    def decode(self, state, bitstream, state_orig):
        """decode the entire bitstream, given the initial state
           NOTE: the bitstream is decoded right to left

        Args:
            state (int): the initial state of the decoding process, in the range [0, L)
            bitstream (list): the bitstream to decode

        Returns:
            list: list of decoded symbols, in the order they were decoded
        """
        
        # initialize empty list to store decoded symbols
        decoded = []
        
        # iterate over the bitstream, decoding each symbol
        # stops when the bitstream is empty and the state is equal to the original state
        # note: we do state_orig - self.L because the encoding table is from L to 2L
        # and we decoding is from 0 to L
        while len(bitstream) > 0 or state != (state_orig - self.L):
            # decode a single symbol
            s_decode, state, bitstream = self.decode_step(state, bitstream)
            # append the decoded symbol to the list
            decoded.append(self.s_list[s_decode])
            #print(state)
            
        # return the list of decoded symbols
        return decoded

In [3]:
# encoding
class Encoder:
    def __init__(self, L, s_list, L_s, spread):
        """Initializes the encoder

        Args:
            L (int): the sum of frequencies, also the length of the table
            s_list (list): list of symbols in the message
            L_s (list): list of frequencies of the symbols
            spread (list): spread of the symbols
        """
        self.L = L
        self.s_list = s_list
        self.L_s = L_s

        # setup the table        
        self.table = [0 for i in range(L)]
        
        # spread symbols using the spread function
        self.symbol_spread = spread
        
        # setup other variables
        self.k = None
        self.nb = None
        self.start = None
        
        # setup the encoder
        self.setup()
        
    def display_table(self):
        """Generates a pandas dataframe to display the encoding table

        Returns:
            pd.DataFrame: a pandas dataframe with the encoding table
        """
        table = []
        for i in range(self.L):
            table.append(self.table[i])
            
        df = pd.DataFrame({"Next": table,"State": [i for i in range(self.L, self.L*2)]})
        df.set_index("State", inplace=True)
        return df.T
    
    @staticmethod
    def use_bits(num, n_bits):
        """Uses the first n_bits LSB from num, and returns the bits and the remaining number after removing the bits.

        Args:
            num (int): the number to extract bits from
            n_bits (int): number of bits to extract

        Returns:
            (list, int): returns (bits_list, num) where bits_list is the list of bits extracted, and num is the remaining 
            number after removing the bits
        """
        # get the first n_bits (LSB) from num
        if n_bits == 0:
            return [], num
        bits = num & ((1 << n_bits) - 1)
        num >>= n_bits
        # convert bits to a list of 0s and 1s
        bits_list = [int(bit) for bit in bin(bits)[2:].zfill(n_bits)]
        return bits_list, num
     
    def setup(self):
        """Sets up the encoding table, calculates k, nb, start, and next values

        Returns:
            list: returns the encoding table list
        """
        
        # calculate R = log2(L), the number of bits needed to represent L
        R = int(math.log2(self.L))
        # calculate r = R + 1
        r = R + 1
        
        # define k, which is the number of bits needed to represent each symbol
        self.k = [R - math.floor(math.log2(self.L_s[i])) for i in range(len(self.L_s))]
        
        # define nb
        self.nb = [(self.k[i] << r) - (self.L_s[i] << self.k[i]) for i in range(len(self.L_s))]
        
        # define start
        self.start = [0 for i in range(len(self.L_s))]
        for i in range(len(self.L_s)):
            self.start[i] = -self.L_s[i] + sum([self.L_s[j] for j in range(i)])
        
        # define next
        next = self.L_s.copy()
        
        # generate the encoding table
        for i in range(self.L):
            s = self.symbol_spread[i]
            self.table[self.start[s] + next[s]] = i + self.L
            next[s] += 1
            
        return self.table
    
    def encode_step(self, state, s):
        """A single step in the encoding process, encodes a single symbol

        Args:
            state (int): the current state of the encoding process, should be in the range [L,2L)
            s (void): the symbol to encode, can be of any type, but must be in the s_list

        Returns:
            (list, int): returns (bitstream, next state) where bitstream is the list of bits encoded, and next state is the next state
        """
        
        r = int(math.log2(self.L * 2))
        
        nbBits = (state + self.nb[s]) >> r
                
        bitstream, state = Encoder.use_bits(state, nbBits)
        
        state = self.table[self.start[s] + (state)]
        
        return bitstream, state
    
    def encode(self, data):
        """Encodes the entire data, and returns the bitstream

        Args:
            data (list): a list of symbols to encode, all symbols should be in the s_list

        Returns:
            (list, int): returns (bitstream, state) where bitstream is the list of bits encoded, and 
            state is the final state in [L,2L)
        """
        bitstream = []
        state = 0
        
        # initialize the state
        bits, state = self.encode_step(state, self.s_list.index(data[0]))
        
        # save original state
        state_orig = state
        
        for s in data:
            s_orig = s
            s = self.s_list.index(s)
            bits, state = self.encode_step(state, s)
            bitstream.extend(bits)
            
        # encode the final state
        #bitstream.extend(Encoder.use_bits(state - self.L, int(math.log2(self.L)))[0])
            
        return bitstream, state, state_orig

In [4]:
class Coder:
    def __init__(self, L, s_list, L_s, fast = False):
        """Initializes the coder

        Args:
            s_list (list): list of symbols in the message
            L_s (list): list of frequencies of the symbols
        """
        self.s_list = s_list
        self.L_s = L_s
        self.L = L
                        
        # initialize the decoding table
        self.decoding_table = DecodeTable(self.L, s_list, L_s, fast= fast)
        
        # spread symbols using the spread function
        self.spread = self.decoding_table.symbol_spread
        
        # initialize the encoder
        self.encoder = Encoder(self.L, s_list, L_s, self.spread)
        
    def encode(self, data):
        """Encodes the data using the encoder

        Args:
            data (list): a list of symbols to encode

        Returns:
            list: returns the final state in [0,L) and the bitstream
        """
        return self.encoder.encode(data)[1] - self.L, self.encoder.encode(data)[0], self.encoder.encode(data)[2]
    
    def decode(self, state, bitstream, orig_state):
        """Decodes the bitstream using the decoding table

        Args:
            state (int): the initial state of the decoding process
            bitstream (list): the bitstream to decode

        Returns:
            list: returns the decoded symbols
        """
        # note we reverse the decoded symbols since the bitstream is decoded in opposite order
        res = self.decoding_table.decode(state, bitstream, orig_state)
        res.reverse()
        return res
    
    @staticmethod
    def calculate_bits(number):
        if number == 0:
            return 1  # Special case for zero
        num_bits = 0
        while number:
            number >>= 1  # Right shift by 1 bit
            num_bits += 1
        return num_bits
    
    def encode_decode(self, data, verbose = False):
        """Encodes and decodes the data, and returns the decoded data

        Args:
            data (list): a list of symbols to encode and decode

        Returns:
            list: returns the decoded symbols
        """
        
        state, bitstream, orig_state = self.encode(data)
        res = self.decode(state, bitstream, orig_state)
        # if there is an error 
        if res != data:
            print("Error in encoding and decoding")
            
        # compute how many bits saved
        orig_bits = len(data) * Coder.calculate_bits(len(self.s_list))
        comp_bits = len(bitstream)
        
        if verbose:
            print("Original bits: %d, Compressed bits: %d, Saved: %d" % (orig_bits, comp_bits, orig_bits - comp_bits))
        
        return res, comp_bits
    
    def encode_decode_string(self, data):
        """Encodes and decodes the string data, and returns the decoded data

        Args:
            data (str): a string to encode and decode

        Returns:
            list: returns the decoded symbols
        """
        data_t = list(data)
        print(data_t)
        return "".join(self.encode_decode(data_t)[0])

# Importing The Symbol Table

In [5]:
s_tabs = [ pd.read_csv(f"trace/input_{i}_flat.apack", sep = " ", header = None) for i in range(8)]

for s_tab in s_tabs:            
    s_tab.columns = ["vmin","off","abits","obits","vcnt"]

s_tabs[0]

,vmin,off,abits,obits,vcnt
0,0,1,127219,1264065,1264065
1,2,5,352906,453435,90687
2,32,4,0,0,0
3,48,4,0,0,0
4,64,4,0,0,0
5,80,4,0,0,0
6,96,4,0,0,0
7,112,4,0,0,0
8,128,4,0,0,0
9,144,4,0,0,0


# Importing the data

In [6]:
# importing the data
data = [np.load(f"trace/input_{i}.npy") for i in range(8)]
#data_flat = data.flatten()
np.unique(data[0]), data[0].shape

(array([0, 1, 2, 3, 4, 5], dtype=int8), (9, 1, 3, 224, 224))

# Compressing the data


1. Convert each data point to a *(symbol, offset)* pair
2. Compress Values to value stream
3. Put offset in offset stream

### *(Symbol, Offset)* Pair

In [7]:
import bisect

class CompPoint:
    def __init__(self, point, s_tab):
        self.s_tab = s_tab
        self.point = point
        self.symbol = self.get_symbol()
        self.off = self.get_off()
        
    def __str__(self):
        return "Point: %d, Symbol: %d, Offset: %d" % (self.point, self.symbol, self.off)
        
    def get_symbol(self):
        # Use binary search to find the symbol
        vmin = self.s_tab["vmin"]
        idx = bisect.bisect_left(vmin, self.point)
        return idx - 1 if idx > 0 else 0
    
    def get_off(self):
        return self.point - self.s_tab["vmin"][self.symbol]

class CompTensor:
    def __init__(self, tensor, s_tab):
        self.tensor = tensor
        self.shape = tensor.shape
        self.flat = tensor.flatten()
        
        # get the symbols and offsets for each point (as a list of CompPoints)
        self.points = [CompPoint(point, s_tab) for point in self.flat]

    def update_tensor(self):
        # update tensor with new values from self.flatten
        self.tensor = self.flatten.reshape(self.shape)
        
    def update_flatten(self):
        # update flatten with new values from self.tensor
        self.flat = self.tensor.flatten()
        
    def to(self, type):
        self.tensor = self.tensor.to(type)
        self.update_flatten()
        
        return self

In [9]:
# converting each data point to a symbol, offset pair

comp_tensors = []
for i, dat in enumerate(data):
    print(f"Processing data {i}")
    comp_tensors.append([CompTensor(d,s_tabs[i]) for d in dat])

# Access points of the first CompTensor of the first sublist
print(comp_tensors[0][0].points[0])

Processing data 0
Processing data 1
Processing data 2
Processing data 3
Processing data 4
Processing data 5
Processing data 6
Processing data 7
Point: 0, Symbol: 0, Offset: 0


In [15]:
str(comp_tensors[4][2].points[2])

'Point: 7, Symbol: 1, Offset: 5'

### Compress Symbols to Stream

In [16]:
def next_power_of_2(x):
    return 1 if x == 0 else 2**(math.ceil(math.log2(x)))

def rescale_list_to_power_of_2(input_list):
    current_sum = sum(input_list)
    if current_sum == 0:
        raise ValueError("The sum of the list elements is zero, cannot rescale.")
    
    # Find the nearest power of 2 greater than or equal to the current sum
    nearest_power_of_2 = next_power_of_2(current_sum)
    
    # Calculate the scaling factor
    scaling_factor = nearest_power_of_2 / current_sum
    
    # Scale the list elements and round them to integers
    scaled_list = [int(round(x * scaling_factor)) for x in input_list]
    
    # Adjust the sum to be exactly the nearest power of 2
    difference = nearest_power_of_2 - sum(scaled_list)
    
    # Adjust the elements to ensure the sum is correct
    for _ in range(abs(difference)):
        if difference > 0:
            # Find the index to increment
            index_to_adjust = scaled_list.index(min(scaled_list))
            scaled_list[index_to_adjust] += 1
        elif difference < 0:
            # Find the index to decrement
            index_to_adjust = scaled_list.index(max(scaled_list))
            scaled_list[index_to_adjust] -= 1
    
    return scaled_list

def rescale_list_to_multiple_of_2(input_list):
    current_sum = sum(input_list)
    if current_sum == 0:
        raise ValueError("The sum of the list elements is zero, cannot rescale.")

    # Find the nearest multiple of 2 greater than or equal to the current sum
    nearest_multiple_of_2 = (current_sum + 1) // 2 * 2
    
    # Calculate the scaling factor
    scaling_factor = nearest_multiple_of_2 / current_sum
    
    # Scale the list elements and round them to integers
    scaled_list = [int(round(x * scaling_factor)) for x in input_list]
    
    # Adjust the sum to be exactly the nearest multiple of 2
    difference = nearest_multiple_of_2 - sum(scaled_list)
    
    # Adjust the elements to ensure the sum is correct
    for _ in range(abs(difference)):
        if difference > 0:
            # Find the index to increment
            index_to_adjust = scaled_list.index(min(scaled_list))
            scaled_list[index_to_adjust] += 1
        elif difference < 0:
            # Find the index to decrement
            index_to_adjust = scaled_list.index(max(scaled_list))
            scaled_list[index_to_adjust] -= 1
    
    return scaled_list

In [17]:
# Getting freqs, must be a power of 2
freqs = []

for s_tab in s_tabs:
    # Get frequencies from the symbol table
    freq = list(s_tab.vcnt)

    # If any of the frequencies are 0, make them 1
    freq = [1 if f == 0 else f for f in freq]

    # rescale to a power of 2 so it works with ANS
    freq = rescale_list_to_power_of_2(freq)
    
    # append to freqs
    freqs.append(freq)

    print("Sum:", sum(freq))

Sum: 2097152
Sum: 8388608
Sum: 8388608
Sum: 2097152
Sum: 262144
Sum: 8192
Sum: 8192
Sum: 16384


In [19]:
import time
nbits = 4 # takes 4 bits to represent each symbol

for i in range(4, len(freqs)):
    print(f"Compressing Layer {i}")

    times = []
    comp_ratios = []
    bps = []

    for j in range(len(comp_tensors[i])):
        
        c = Coder(sum(freqs[i]), [i for i in range(len(freqs[i]))], freq, fast = False)

        msg = [p.symbol for p in comp_tensors[i][j].points]

        time_start = time.time()
        out, comp_bits = c.encode_decode(msg)
        time_end = time.time()
        time_taken = time_end - time_start

        if out != msg:
            print("Error in encoding and decoding")
            break
        
        times.append(time_taken)
        comp_ratios.append(len(msg) * nbits / comp_bits)
        bps.append(comp_bits / time_taken)
        
    # print average stats
    print("\tAverage time taken: %f seconds" % np.mean(times))
    print("\tAverage compression ratio: %f" % np.mean(comp_ratios))
    print("\tAverage bits per second: %f" % np.mean(bps))

Compressing Layer 4
